### This notebook focuses on visualization of accident risk.

In [1]:
# Imports and helper functions
import json
import networkx as nx
import numpy as np
import operator
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from matplotlib import pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

def plot_line(ax, ob, color):
    x, y = ob.xy
    ax.plot(x, y, color=color, linewidth=1, zorder=1)
    
def recurse(d, maximum, minimum, key):
    if type(d)==dict:
        for k,v in d.items():
            if (k==key):
                maximum, minimum = recurse(d[k], maximum, minimum, key)
    else:
        if (d>maximum):
            maximum=d
            return maximum, minimum
        if (d<minimum):
            minimum = d
            return maximum, minimum
    return maximum, minimum

In [2]:
# Load in the network
network = nx.read_gpickle('/home/louisf/Documents/Insight/massdriver/notebooks/graph_with_risk2.pickle')

In [ ]:
# Plot a map using the edges of the network to define line segments. Use 'acc_risk' to color the edges

In [3]:
lines = {}

it=0
for u, v in network.edges_iter():
    js = network[u][v]['Json']
    coords = json.loads(js)['coordinates']
    line = LineString(coords)
    lines[it] = {'line':line, 'rate':network[u][v]['acc_risk']}
    it+=1
    

In [18]:
maximum = 0
minimum = 0
all_rates = []

### HACK
maxval = np.std(all_rates)*10

for k,v in lines.items():
    rate = v['rate']
    if rate>maximum:
        maximum=rate
    if rate<minimum:
        minimum=rate
        
maxval = np.std(all_rates)*10
for k, v in lines.items():
    adj_rate = rate-minimum
    if adj_rate > maxval:
        adj_rate = maxval
    v['scaled_rate']=np.log10(rate-minimum+1)/np.log10(-minimum+maximum+1)
    v['adj_rate'] = adj_rate

KeyboardInterrupt: 

In [21]:

maxval = np.std(all_rates)*10
for k, v in lines.items():
    adj_rate = rate-minimum
    if adj_rate > maxval:
        adj_rate = maxval
    v['scaled_rate']=np.log10(rate-minimum+1)/np.log10(-minimum+maximum+1)
    v['adj_rate'] = adj_rate

In [26]:
# This generates the rate-coded map. This is slow!

colormap = plt.get_cmap('jet')
fig = plt.figure(figsize=(20,10))
ax = plt.subplot(1,1,1)
for k, v in lines.items():
    plot_line(ax, v['line'], colormap(log(v['adj_rate']))
fig.savefig('rescaled2.png', dpi=300, transparent=True)

SyntaxError: invalid syntax (<ipython-input-26-ca12f682bd23>, line 8)

In [ ]:
from osgeo import ogr, osr
import networkx as nx
from matplotlib import pyplot as plt
%matplotlib inline

driver = ogr.GetDriverByName('ESRI Shapefile')
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
dataset = driver.Open(filepath)

# from Layer
layer = dataset.GetLayer()
spatialRef = layer.GetSpatialRef()
# from Geometry
feature = layer.GetNextFeature()
geom = feature.GetGeometryRef()
spatialRef = geom.GetSpatialReference()

shp = ogr.Open(filepath)

In [ ]:
g = read_shp2(filepath, ['RoadSegmen', 'AssignedLe'], simplify=True)

In [ ]:
first = g.edges()[0]
g.edge[(-71.10891187098966, 42.474431813774984)][(-71.10750084168293, 42.47446692343721)]['AssignedLe']

In [ ]:
g2 = g
nx.convert_node_labels_to_integers(g, label_attribute="oldLabels")

In [ ]:
g.node

In [ ]:
it = 0
for n in g.nodes_iter():
    it+=1
    if (it%100 == 0):
        print(n)

In [ ]:
n = g.nodes()

p = nx.shortest_path(g, source=n[0], target = n[3])
len(p)

In [ ]:
import numpy as np
import scipy.spatial.distance as distance

In [ ]:
it=0
lat = 42.198
long = -70.9987
ar = np.empty((0, int(graph.net.number_of_nodes())))
it = 0 
for longitude, latitude in graph.net.nodes_iter():
    ar = np.append(ar, [[longitude, latitude]], axis=0)
    it+=1

In [ ]:
graph.net.number_of_nodes()

In [ ]:
from scipy.spatial.distance import cdist
a = np.array([[long, lat]])
others = np.asarray(graph.net.nodes())
def closest_point(pt, others):
    distances = cdist(pt, others)
    return distances.argmin()
closest_point(a, others)

In [ ]:
np.asarray(others)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon

In [ ]:
import graphHandler as gh


graph = gh.NetworkGenerator()
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
graph.loadGraph(filepath=filepath, fields=['RoadSegmen', 'AssignedLe'], simplify=True)

In [ ]:
lat1 = 42.198
long1 = -70.9987
start = gh.findClosestNode(graph.net, lat1, long1)
end = gh.findClosestNode(graph.net, lat1+.035, long1+.006)

In [ ]:
path = gh.getShortestPath(graph.net, tuple(start), tuple(end))

In [ ]:
import pickle


with open('graph.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(graph, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
b=np.asarray(path)

In [ ]:
np.reshape(b.flatten(), (len(b),2))

In [ ]:
b

In [ ]:
lat1 = 42.198
long1 = -70.9987
lat2 = 42.298
long2 = -70.81
path = gh.pathingSolution(graph.net, lat1, long1, lat2, long2, 'AssignedLe')

In [ ]:
import numpy as np
a = np.array([[1, 1], [2,3]])

In [ ]:
import networkx as nx
import graphHandler as gh
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
graph = gh.NetworkGenerator()
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
graph.loadGraph(filepath=filepath, fields=['RoadSegmen', 'AssignedLe'], simplify=True)

In [ ]:
pred_arr = np.load('data_with_prediction.npy')

In [ ]:
it=0
for u,v in graph.net.edges_iter():
    segnum = graph.net[u][v]['RoadSegmen']
    #rowid = pred_arr[:,48]==segnum
    try:
        rowid = np.nonzero(pred_arr[:,48]==segnum)[0][0]
        graph.net[u][v]['acc_risk'] = pred_arr[rowid,-1]
    except IndexError:
        graph.net[u][v]['acc_risk'] = 0
    
    it+=1

In [ ]:
nx.write_gpickle(graph.net, 'graph_with_risk2.pickle')

In [ ]:
segnum

In [ ]:
a = []
for u,v in graph.net.edges_iter():
    a.append(graph.net[u][v]['acc_risk'])

In [ ]:
np.nonzero(pred_arr[:,48]==segnum)[0][0]